<a href="https://colab.research.google.com/github/AkashR04/CUAD-summarization/blob/main/Summarization_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contract summarization for cuad dataset

## Dependencies

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## BART summarization


### Pre-processing

In [6]:
from transformers import AutoTokenizer

checkpoint = "ainize/bart-base-cnn"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = """AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

"""text = tokenizer.convert_ids_to_tokens(inputs.input_ids)

text"""


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    0,  2620,  9309, 12613,  8228,     4,   112,  3842,  4062, 41548,
         11839,   108,   104, 29892,  2068, 14426, 12866,  7111,   104,     6,
          2291,     8, 13091,  7539,  2076,   234,  6972,   500, 25815, 19328,
         35473,  6034, 12408, 29457, 13709,  2492,  1862,  5680,  4629, 20944,
         50118, 50118,   713,  8352,   440,     4,   112,    36,   627,    22,
         10127, 40720, 16844,  2867,    88,    30,     8,   227,   312,  5593,
          1758,    18,  8784,  8465,     6,  2291,  6697, 13584, 37061,  8070,
          2275, 20056,  2824,  6697, 35132,   906,  8070,    16,  2375,    25,
             9,   502,   365,     6,   199,  6697, 10127, 40720, 33355, 10566,
          8070,     8,   524,  8845,    14,  1402, 12047,  8759,   227,  8784,
             8,  2275,  7000,   550,   112,     6,   193,  6697, 19897, 43563,
         18653,  8784,     8,  2275,    32,   349,  4997,     7, 21887,    25,
            10,    22, 38210,   113,  

ValueError: ignored

### Loading model

In [3]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

model

Some weights of the model checkpoint at ainize/bart-base-cnn were not used when initializing BartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BartModel(
  (shared): Embedding(50265, 768, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): Embedding(50265, 768, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
    (layers): ModuleList(
      (0): BartEncoderLayer(
        (self_attn): BartAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): BartEncoderLayer(
        (sel

### Example Summary

In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model = "ainize/bart-base-cnn")
summary = summarizer(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'summary_text': "Stremick's Heritage Foods, LLC and Premier Nutrition Corporation have agreed to amend their Manufacturing Agreement .\nThe Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following .\nAll other terms, conditions and covenants of the Agreement shall remain in full force and effect ."}]


## Pegasus summarization

### Pre-processing

In [1]:
#from transformers import AutoTokenizer

checkpoint = "google/pegasus-billsum"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = """AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

"""text = tokenizer.convert_ids_to_tokens(inputs.input_ids)

text"""


{'input_ids': tensor([[ 3887, 24426, 18106,  4946,   107,   305,  2824, 32482, 64342, 18081,
           131,   283, 25063,  7542, 17543, 41154,   283,   108,  3936,   111,
         24996,   897, 26861, 24332, 13118, 48616, 82220, 21693,  2252, 42196,
         55947, 74069,   182, 11744,   566,   107,   305,   143,   544,   198,
         74867,   194,   312,  3295,   190,   141,   111,   317, 45466, 63920,
           131,   116,  6269, 10992,   108,  3936, 10528, 65405,  7910,  6235,
         11136,  4356, 10528, 45534,  7910,   117,   957,   130,   113,  1185,
          7320,   931, 10528, 74867, 16754,  5613,  7910,   111, 54802,   120,
           878, 10513,  6427,   317,  6269,   111,  6235,  8922,  1307,  2628,
          1326, 10528, 79204, 16743,  6269,   111,  6235,   127,   276,  3795,
           112, 15600,   130,   114,   198, 29297,   194,   111, 12896,   130,
           109,   198, 91053,   496, 40297, 10536,   108,  6269,   111,  6235,
          3295,   190,   109,  6427,  

'text = tokenizer.convert_ids_to_tokens(inputs.input_ids)\n\ntext'

### Loading model

In [4]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

model

Some weights of the model checkpoint at google/pegasus-billsum were not used when initializing PegasusModel: ['final_logits_bias']
- This IS expected if you are initializing PegasusModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PegasusModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of PegasusModel were not initialized from the model checkpoint at google/pegasus-billsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PegasusModel(
  (shared): Embedding(96103, 1024, padding_idx=0)
  (encoder): PegasusEncoder(
    (embed_tokens): Embedding(96103, 1024, padding_idx=0)
    (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
    (layers): ModuleList(
      (0): PegasusEncoderLayer(
        (self_attn): PegasusAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): ReLU()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): Pegasus

### Example Summary

In [ ]:
from transformers import pipeline
summarizer2 = pipeline("summarization", model = "google/pegasus-billsum")
summary = summarizer2(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

[{'summary_text': '(This measure has not been amended since it was reported to the House on July 9, 2018. (Sec. 1) This bill amends the Manufacturing Agreement between Heritage Foods, LLC, and Premier Nutrition Corporation (the parties to the manufacturing agreement) to revise the terms and conditions of the agreement.'}]


## T5 Summarization

In [5]:
from transformers import AutoTokenizer

checkpoint = "stevhliu/t5-small-finetuned-billsum-ca_test"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = """AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

"""text = tokenizer.convert_ids_to_tokens(inputs.input_ids)

text"""


{'input_ids': tensor([[ 5422, 14920, 11810,  5693,     5,   209,  3001,  5097, 26051, 25286,
            31,   134,     3, 17444,  3177, 11187, 12408,  7039,   134,     6,
          6516,    11, 22694,   329, 18988, 13046, 16840,  9562,  2847,  6294,
          2990,  8015,     3,  9312, 18899,  9262,  5905,  2365,  8859,  4386,
         21221,   100, 18397,   465,     5,   209,    41,   532,    96,  8123,
           989,   297,  8512,     6,  5136,   139,    57,    11,   344,   472,
            60,    51,  3142,    31,     7, 11523,  3139,     7,     6,  6516,
         13308, 12636,   155,   545,  8512,  6552, 16525,  6708, 13308, 10572,
            51,   972,  8512,    19,  1231,    38,    13,  1515,  9694,   846,
         13308,  8123,   989,   297, 18652,  7678,  8512,    11, 14545,     7,
            24,   824, 19193,  7139,   344, 11523,    11,  6552,     3, 14134,
          1718,  1914,  1233, 13308,   188,  3584,  1194,   121,   137, 11523,
            11,  6552,    33,   284,  

'text = tokenizer.convert_ids_to_tokens(inputs.input_ids)\n\ntext'

### Loading model

In [6]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

model

Some weights of the model checkpoint at stevhliu/t5-small-finetuned-billsum-ca_test were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dropout(p=0.1, inplace=

### Example Summary

In [ ]:
summarizer3 = pipeline("summarization", model = "stevhliu/t5-small-finetuned-billsum-ca_test")
summary = summarizer3(
    """
AMENDMENT NO. 1 TO STREMICK'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT

This Amendment No. 1 (the "Amendment"), entered into by and between Stremick's Heritage Foods, LLC ("Heritage") Premier Nutrition Corporation ("Premier") is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). Heritage and Premier are each referred to herein as a "Party" and collectively as the "Parties."

WHEREAS, Heritage and Premier entered into the Agreement;

WHEREAS, the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

NOW, THEREFORE, in consideration of the promises and of the mutual covenants, representations and warranties contained in the Agreement and set forth herein, the Parties hereby agree that the following changes shall be made to the Agreement:

1. The Parties hereby agree to remove Section 2(c) in its entirety and replace it with the following:

During the Term of this Agreement, Premier shall be required to purchase a Minimum Annual Order Volume ("MAOV") of [***] ("Units") for the twelve-month period commencing July 1, 2018 and ending June 30, 2019. The MAOV [***] Units for each twelve-month period commencing July 1, 2019 through the end of the Term, contingent upon commercial aseptic production at Heritage's [***] facility and approval of that facility by Premier by January 1, 2019. For the avoidance of doubt, the six-month period commencing July 1, 2022 and ending December 31, 2022, Premier will be required to purchase [***] (the twelve-month periods and the six-month period are each a "Contract Period").

2. Except as otherwise specified above in this Amendment, all other terms, conditions and covenants of the Agreement shall remain in full force and effect.

IN WITNESS WHEREOF, the Parties have caused this Amendment to be signed by their respective duly authorized representatives as of the Amendment Effective Date. Premier Nutrition Corporation Stremick's Heritage Foods, LLC.

By: /s/ Darcy Davenport By: /s/ Sam Stremick Name: Darcy Davenport Name: Sam Stremick Title: President Title: President

Source: BELLRING BRANDS, INC., S-1, 9/20/2019
"""
)

print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'summary_text': 'AMENDMENT NO. 1 TO STREMICK\'S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT This Amendment is effective as of June 11, 2018 ("Amendment Effective Date") and amends that certain Manufacturing Agreement between Heritage and Premier dated July 1, 2017 ("Agreement"). The amendment would require the Parties to remove Section 2(c) in its entirety and replace it with the following changes shall be made to the Agreement. This amendment would authorize the Partie to amend the Agreement in accordance with the terms and conditions set forth herein. This Amendment, signed by and between Stremick\'s Heritage Foods, LLC ("Heritage") is effective on June 11, 2018, and amend that certain manufacturing Agreement between and between Premier Nutrition Corporation ("Premier"'}]
